# Prepare cost raster

This notebook is used to prepare the cost rasters required to run the updated grid routing algorithm developed for DRC. The cost raster divides the country into raster cells of a chosen resolution (e.g. 50-200m). Each raster cell represents the "cost" of traversing it while constructing a new grid line, where *1* is the best cost, and raster cells with a higher value are harder (more expensive) to cross. 

The final cost layer is made up from the combination of 5 different costs layers, based on a review of the existing literature. These layers are:
* Roads (it's easiest to construct new grid lines along existing roads)
* Rivers (it's harder to cross rivers)
* Water bodies (it's harder to cross water bodies)
* Slope (it's harder to construct power lines at steep slopes)
* Power lines (outlining the existing power lines)



#### First import the neccessary packages

In [ ]:
import fiona
import rasterio
import rasterio.mask
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
import json
import richdem as rd
from shapely.geometry import box
from shapely.geometry import shape, mapping
import pandas as pd
import numpy as np
from geopandas import clip
import scipy
import scipy.spatial
import datetime
import numba
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from osgeo import gdal
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

%matplotlib inline
%load_ext line_profiler

import warnings
warnings.filterwarnings('ignore')

#### Define the workspace to save data in

In [ ]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')

workspace = filedialog.askdirectory()

#### Select the grid you are working with

Must be named **'Est'**, **'Sud'** or **'Ouest'**

In [ ]:
grid = 'Ouest'

#### Define the target coordinate reference system (CRS) - should be in meters

In [ ]:
crs = 'EPSG:3395'

#### Next, open the polygon of your study area (shapefile)

The national boundary can be retrieved for countries from e.g. https://gadm.org/.
Note that for DRC, we create the cost raster for the three areas, extending e.g. 50 km from the existing grid lines, rather than for the whole country, in order toreduce computational time. These boundaries are found in the **gis_data** folder for the three grids

In [ ]:
messagebox.showinfo('OnSSET', 'Open the polygon of the study area of the selected grid')
boundaries_path = filedialog.askopenfilename()

boundaries = gpd.read_file(boundaries_path).to_crs(crs)

#### In the next step, we create a raster of the study

First, define the resolution. Then run the second cell as well just as it is.

In [ ]:
resolution = 50 # meters (same unit as your crs)

In [ ]:
total_bounds = boundaries['geometry'].total_bounds
width = round((total_bounds[3] - total_bounds[1])/resolution)
height = round((total_bounds[2] - total_bounds[0])/resolution)

path =  workspace + '/' + grid + '_raster.tif'

shape =  height, width
transform = rasterio.transform.from_bounds(*boundaries['geometry'].total_bounds, shape[0], shape[1])
rasterized = rasterize(
    [(shape) for shape in boundaries['geometry']],
    out_shape=(width, height),
    transform=transform,
    all_touched=True,
    dtype=rasterio.uint8)

try:
    raster.close()
    os.remove(path)
except (FileNotFoundError, NameError):
    pass

with rasterio.open(
    path, 'w',
    driver='GTiff',
    dtype=rasterio.uint8,
    count=1,
    crs = crs,
    width=shape[0],
    height=shape[1],
    transform=transform,
    nodata = 0
) as dst:
    dst.write(rasterized, indexes=1)
    
raster = rasterio.open(path)

shape = raster.shape
affine = raster.transform

out_meta = raster.meta

out_meta.update({"driver": "GTiff",
                 "height": raster.height,
                 "width": raster.width,
                 "transform": raster.transform,
                 'compress': 'NONE',
                 'dtype': rasterio.float32,
                 "crs": raster.crs,
                 'nodata': 9999})

#### Create cost layer from roads

Roads with the correct attributes can be retrieved from Open Street Map through Geofabrik: https://download.geofabrik.de/africa/congo-democratic-republic.html 

In [ ]:
messagebox.showinfo('OnSSET', 'Open the roads shapefile for the country')
roads_path = filedialog.askopenfilename()

roads = gpd.read_file(roads_path, mask=boundaries).to_crs(crs)

In [ ]:
roads["weight"] = 99

# Here you define the weight of different types of roads. Lower values are more favorable for grid extension.
roads.loc[roads["fclass"] == "motorway", "weight"] = 1
roads.loc[roads["fclass"] == "trunk", "weight"] = 1
roads.loc[roads["fclass"] == "primary", "weight"] = 1
roads.loc[roads["fclass"] == "secondary", "weight"] = 1 
roads.loc[roads["fclass"] == "tertiary", "weight"] = 2
roads.loc[roads["fclass"] == "unclassified", "weight"] = 2
roads.loc[roads["fclass"] == "residential", "weight"] = 2
roads.loc[roads["fclass"] == "service", "weight"] = 2

roads = roads[roads.weight != 99]

roads = roads.sort_values(by="weight", ascending=False)

roads_for_raster = [(row.geometry, row.weight) for _, row in roads.iterrows()]

roads_raster = rasterize(
        roads_for_raster,
        out_shape=shape,
        fill=99,
        default_value=99,
        all_touched=True,
        transform=affine,
    )

#with rasterio.open(workspace + '/' + grid + '_roads_cost.tif', 'w', **out_meta) as dst:
#    dst.write(roads_raster, indexes=1)

#### Create cost layer from power lines

In [ ]:
messagebox.showinfo('OnSSET', 'Open the existing grid shapefile for the country')
power_path = filedialog.askopenfilename()

power = gpd.read_file(power_path, mask=boundaries).to_crs(crs)

In [25]:
# The weight of the power raster should be set to 0, indicating that this is the starting point for the new grid extension algorithm
power['weight'] = 0

power_for_raster = [(row.geometry, row.weight) for _, row in power.iterrows()]

power_raster = rasterize(
        power_for_raster,
        out_shape=shape,
        fill=999,
        default_value=0,
        all_touched=True,
        transform=affine,
    )

with rasterio.open(workspace + '/' + grid + '_power_cost.tif', 'w', **out_meta) as dst:
    dst.write(power_raster, indexes=1)

#### Create cost layer from water bodies (lakes etc.)

Water bodies can be retrieved e.g. from https://energydata.info/dataset/africa-water-bodies-2015

In [ ]:
messagebox.showinfo('OnSSET', 'Open the water bodies shapefile for the country')
waters_path = filedialog.askopenfilename()

waters = gpd.read_file(waters_path, mask=boundaries).to_crs(crs)

In [ ]:
# Here you define the weight of water bodies
waters['weight'] = 99

waters_for_raster = [(row.geometry, row.weight) for _, row in waters.iterrows()]


if len(waters_for_raster) > 0:
    waters_raster = rasterize(
            waters_for_raster,
            out_shape=shape,
            fill=0,
            default_value=0,
            all_touched=True,
            transform=affine,
        )
else:
    waters_raster = power_raster * 0

#with rasterio.open(workspace + '/' + grid + '_waters_cost.tif', 'w', **out_meta) as dst:
#    dst.write(waters_raster, indexes=1)

#### Create cost layer from elevation/slope

Elevation data can be retrieved from https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4 or https://www.diva-gis.org/gdata for V3 SRTM data by country.

This dataset is then used to create the slope layer.
First, select the elevation (DEM) layer. Next, define the weights assigned to different slopes.

In [ ]:
messagebox.showinfo('OnSSET', 'Open the elevation raster for the grid region')
dem_path = filedialog.askopenfilename()

In [ ]:
slope_weights = {5: 1,       # I.e. all slopes less than 5 degrees have a multiplier of 1
                 10: 1.25,    # I.e. all slopes between 5 and 10 degrees have a multiplier of 1.5
                 20: 1.5,
                 9999: 2}

In [ ]:
def processing_elevation_and_slope(workspace, crs, dem_path):
    raster=rasterio.open(dem_path)

    gdal.Warp(workspace + r"\dem.tif", raster.name, dstSRS=crs)

    def calculate_slope(DEM):
        gdal.DEMProcessing(workspace + r'\slope.tif', DEM, 'slope')
        with rasterio.open(workspace + r'\slope.tif') as dataset:
            slope=dataset.read(1)
        return slope

    slope=calculate_slope(workspace + r"\dem.tif")

    slope = rasterio.open(workspace + r'\slope.tif')
    
    return slope
    
slope = processing_elevation_and_slope(workspace, 'EPSG:3395', dem_path)

slope_values = slope.read(1)
slope_meta = slope.meta

destination = np.ones((out_meta['height'], out_meta['width'])) *  999

slope, slope_affine = reproject(
    source=slope_values,
    destination=destination,
    src_transform=slope_meta['transform'],
    src_crs=slope_meta['crs'],
    dst_transform=out_meta['transform'],
    dst_crs=crs,
    resampling=Resampling['nearest'])

with rasterio.open(workspace + '/' + grid + '_slope.tif', 'w', **out_meta) as dst:
    dst.write(slope, indexes=1)

slope_reclassified = np.ones((out_meta['height'], out_meta['width']))

prev_key = 0

for key, value in zip(slope_weights.keys(), slope_weights.values()):
    slope_reclassified = np.where((slope < key) & (slope >= prev_key), value, slope_reclassified)
    prev_key = key
    
#with rasterio.open(workspace + '/' + grid + '_slope_cost.tif', 'w', **out_meta) as dst:
#    dst.write(slope_reclassified, indexes=1)

#### Land cover

Land cover data can be downloaded from https://www.arcgis.com/apps/instant/media/index.html?appid=fc92d38533d440078f17678ebc20e8e2

The ESRI 2021 Land Cover V2 data comes in 10m resolution and has the following classes (available at https://planetarycomputer.microsoft.com/dataset/io-lulc-9-class):
* 0: No data
* 1: Water
* 2: Trees
* 4: Flooded vegetation
* 5: Crops
* 7: Built area
* 8: Bare ground
* 9: Snow/Ice
* 10: Clouds
* 11: Rangeland (Scrub/Shrub)

In [ ]:
messagebox.showinfo('OnSSET', 'Open the land cover raster for the grid region')
land_cover_path = filedialog.askopenfilename()

In [ ]:
# Define the weights for the different land cover types here

land_cover_weights = {0: 3,     # No data cells have a weight of 3
                      1: 20,    # Water cells have a weight of 10
                      2: 4,     # Trees have a weight of 3
                      4: 10,     # ...
                      5: 3,
                      7: 5,
                      8: 3,
                      9: 10,
                      10: 3,
                      11: 3}
                      

In [ ]:
with rasterio.open(land_cover_path) as lc:
        land_cover = lc.read(1)
        land_cover_meta = lc.meta

destination = np.ones((out_meta['height'], out_meta['width'])) *  out_meta['nodata']

lc, lc_affine = reproject(
    source=land_cover,
    destination=destination,
    src_transform=land_cover_meta['transform'],
    src_crs=land_cover_meta['crs'],
    dst_transform=out_meta['transform'],
    dst_crs=crs,
    resampling=Resampling['min'])

lc_reclassified = np.ones((out_meta['height'], out_meta['width']))

for key, value in zip(land_cover_weights.keys(), land_cover_weights.values()):
    lc_reclassified = np.where(lc == key, value, lc_reclassified)
    
#with rasterio.open(workspace + '/' + grid + '_land_cover_cost.tif', 'w', **out_meta) as dst:
#    dst.write(lc_reclassified, indexes=1)

#### Create final cost layer

In [ ]:
costs = np.where(roads_raster < 99, roads_raster, lc_reclassified)
costs = costs * slope_reclassified
costs = np.where(waters_raster == 99, waters_raster, costs)
costs = np.where(rasterized == 0, 9999, costs)

with rasterio.open(workspace + '/' + grid + '_final_cost.tif', 'w', **out_meta) as dst:
    dst.write(costs, indexes=1)